In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/CODESTUDIO-GIT/endtoend-ml-projects/master/50_Startups.csv")

In [3]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


# Feature Engineering 
### 1) Drop the unwanted Columns
### 2) Handle Missing Values
### 3) Handle the Categorical feature
### 4) Handle Feature Scaling
### 5) Remove Outliers

### As per are Analysis in EDA 
### 1) There is no unwanted feature
### 2) There is no missing value
### 3) There is 1 state feature which is categorical which we need to handle 
### 4) There is no outliers in dataset


In [4]:
# handling categorical data
state_dummy = pd.get_dummies(df['State'],prefix='state',drop_first=True)
df = pd.concat([df,state_dummy],axis=1)

In [5]:
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit,state_Florida,state_New York
0,165349.20,136897.80,471784.10,New York,192261.83,0,1
1,162597.70,151377.59,443898.53,California,191792.06,0,0
2,153441.51,101145.55,407934.54,Florida,191050.39,1,0
3,144372.41,118671.85,383199.62,New York,182901.99,0,1
4,142107.34,91391.77,366168.42,Florida,166187.94,1,0


In [6]:
df.drop(['State'],axis=1,inplace=True)

In [7]:
df.head()

,R&D Spend,Administration,Marketing Spend,Profit,state_Florida,state_New York
0,165349.20,136897.80,471784.10,192261.83,0,1
1,162597.70,151377.59,443898.53,191792.06,0,0
2,153441.51,101145.55,407934.54,191050.39,1,0
3,144372.41,118671.85,383199.62,182901.99,0,1
4,142107.34,91391.77,366168.42,166187.94,1,0


# Spliting the Data into Training and Testing Set

In [15]:
X = df.drop(['Profit'],axis=1)
y = df['Profit']

In [14]:
X.head()

,R&D Spend,Administration,Marketing Spend,state_Florida,state_New York
0,165349.20,136897.80,471784.10,0,1
1,162597.70,151377.59,443898.53,0,0
2,153441.51,101145.55,407934.54,1,0
3,144372.41,118671.85,383199.62,0,1
4,142107.34,91391.77,366168.42,1,0


In [17]:
y.head()

0    192261.83
1    191792.06
2    191050.39
3    182901.99
4    166187.94
Name: Profit, dtype: float64

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

# Model Selection

In [19]:
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,ShuffleSplit

In [24]:
# Creating the fucntion for best paramter using Grid search CV
def find_best_model_using_grid_cv(X,y):
    algo = {
        'linear_regression': {
            'model':LinearRegression(),
            'params' : {
                'fit_intercept': [True,False],
                'normalize': [True,False],
                'copy_X': [True,False]
            }
        },
        'lasso':{
            'model':Lasso(),
            'params': {
                'alpha': [1,2],
                'selection':['random','cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algo.items():
        gs = GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_scores': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_scores','best_params'])

In [25]:
pd.set_option('display.max_colwidth',100)
find_best_model_using_grid_cv(X,y)

,model,best_scores,best_params
0,linear_regression,0.922076,"{'copy_X': True, 'fit_intercept': True, 'normalize': True}"
1,lasso,0.922115,"{'alpha': 2, 'selection': 'random'}"
2,decision_tree,0.894233,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


# Model Building

In [26]:
model_lr = LinearRegression(copy_X=True,fit_intercept=True,normalize=True)

In [27]:
model_lr.fit(X_train,y_train)

LinearRegression(normalize=True)

In [29]:
model_lr.score(X_test,y_test)

0.9347068473282423

In [30]:
y_pred = model_lr.predict(X_test)

In [31]:
y_pred

array([103015.20159796, 132582.27760816, 132447.73845174,  71976.09851258,
       178537.48221055, 116161.24230165,  67851.69209676,  98791.73374687,
       113969.43533012, 167921.0656955 ])

In [32]:
X_train.head()

,R&D Spend,Administration,Marketing Spend,state_Florida,state_New York
33,55493.95,103057.49,214634.81,1,0
35,46014.02,85047.44,205517.64,0,1
26,75328.87,144135.98,134050.07,1,0
34,46426.07,157693.92,210797.67,0,0
18,91749.16,114175.79,294919.57,1,0


In [33]:
X.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'state_Florida',
       'state_New York'],
      dtype='object')

In [38]:
X_test.head()

,R&D Spend,Administration,Marketing Spend,state_Florida,state_New York
28,66051.52,182645.56,118148.20,1,0
11,100671.96,91790.61,249744.55,0,0
10,101913.08,110594.11,229160.95,1,0
41,27892.92,84710.77,164470.71,1,0
2,153441.51,101145.55,407934.54,1,0


In [52]:
np.where(X.columns == 'state_'+str('New York'))[0][0]

4

In [45]:
def predict_profit(r_d_spent,Admin_spen,maket_spen,state):
    state_index = np.where(X.columns == 'state_'+str('New York'))[0][0]
    x = np.zeros(len(X.columns))
    x[0] = r_d_spent
    x[1] = Admin_spen
    x[2] = maket_spen
    if state_index > 0:
        x[state_index] = 1
    return model_lr.predict([x])[0]

In [46]:
predict_profit(165349.20,136897.80,471784.10,'New York')

192919.57537462088

In [47]:
import pickle

In [48]:
pickle.dump(model_lr,open("profit_predict_model.pkl",'wb'))

In [51]:
import json

columns = {'data_columns': [col.lower() for col in X.columns]}

with open("columns.json",'w') as f:
    f.write(json.dumps(columns))

In [22]:
d = {"data_columns": ["r&d spend", "administration", "marketing spend", "state_florida", "state_new york"]}

In [26]:
d['data_columns'].index('state_new york')

4